In [1]:
import pandas as pd
import networkx as nx
import numpy as np

In [2]:
graph = "swtor"
max_community_size = 20
#connected_percent = 0.75
#times_average = 1.5
connected_percent = 0.75
times_average = 1.5

In [3]:
print("Reading graph " + str(graph))
graph_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/filtered_triples_weighted/"
graph_df = pd.read_csv(graph_path + graph + ".triples", sep="###", engine="python", header=None)
G = nx.Graph()
for row in graph_df.itertuples():
    G.add_edge(int(row[1]), int(row[2]), weight=int(row[3]))

Reading graph swtor


In [4]:
print(G)
print([n for n in G[0]])

Graph with 18125 nodes and 57118 edges
[2, 2180]


In [5]:
mean_edge_weight = np.mean([weight for _, _, weight in G.edges(data='weight')])
print(mean_edge_weight)

1.3045449770650233


In [6]:
communities_path = "/Users/matevass/Documents/Projects/KGEntityMatching/data/communities/"
communities = list()
to_add = list({i} for i in G.nodes)
to_delete = list()

print("Parameters - connected_percent: " + str(connected_percent) + " - times_average: " + str(times_average))
print("Searching for communities in graph " + graph)

for community_size in range(2, max_community_size + 1):
    print("Community size: " + str(community_size))
    last_iteration = 0
    communities_to_test = to_add.copy()
    to_add.clear()
    to_delete.clear()
    count = 0
    number_to_test = len(communities_to_test)
    print("Number of communities to test: " + str(number_to_test))
    for community in communities_to_test:
        count += 1
        if count % 1000 == 0:
            print(str(count) + "/" + str(number_to_test))
        neighbors = set()
        for node in community:
            neighbors.update(G.neighbors(node))
        neighbors = [item for item in neighbors if item not in community]
        for node in neighbors:
            community_temp = community.copy()
            community_temp.add(node)
            #if community_temp in to_add:
                #continue
            H = G.subgraph(community_temp)
            if len(H[node]) >= (community_size - 1) * connected_percent and \
                    np.mean([weight for _, _, weight in H.edges(data='weight')]) > mean_edge_weight * times_average:
                last_iteration += 1
                if community not in to_delete:
                    to_delete.append(community)
                if set(community_temp) not in to_add:
                    to_add.append(community_temp)
                    
    if community_size == 2:
        communities = [item for item in communities if len(item) != 1]
    if last_iteration == 0:
        print("Break at community size: " + str(community_size))
        break
    
    communities = [item for item in communities if item not in to_delete]
    communities.extend(to_add)
    print("Total communities: " + str(len(communities)))

with open(communities_path + graph + ".csv", 'w') as communities_output:
    for community in communities:
        communities_output.write(str(community)
                                 .replace("{", "")
                                 .replace("}", "")
                                 .replace(", ", ";") + "\n")

print("Communities for graph " + graph + " found")

Parameters - connected_percent: 0.75 - times_average: 1.5
Searching for communities in graph swtor
Community size: 2
Number of communities to test: 18125
1000/18125
2000/18125
3000/18125
4000/18125
5000/18125
6000/18125
7000/18125
8000/18125
9000/18125
10000/18125
11000/18125
12000/18125
13000/18125
14000/18125
15000/18125
16000/18125
17000/18125
18000/18125
Total communities: 16207
Community size: 3
Number of communities to test: 16207
1000/16207
2000/16207
3000/16207
4000/16207
5000/16207
6000/16207
7000/16207
8000/16207
9000/16207
10000/16207
11000/16207
12000/16207
13000/16207
14000/16207
15000/16207
16000/16207
Total communities: 14352
Community size: 4
Number of communities to test: 4564
1000/4564
2000/4564
3000/4564
4000/4564
Total communities: 13387
Community size: 5
Number of communities to test: 675
Total communities: 13707
Community size: 6
Number of communities to test: 909
Total communities: 13505
Community size: 7
Number of communities to test: 353
Total communities: 1336